In [2]:
# =====================================================================================
# Initialize JAX backend
import os
os.environ["KERAS_BACKEND"] = "jax"


# =====================================================================================
# Import modules
import matplotlib.pyplot as plt
import scipy.io as sio
import seaborn as sns
import numpy as np
import sys
from bayesflow.approximators import ContinuousApproximator

# Set up paths for notebook environment
current_dir = os.getcwd()
print(current_dir)

# Add parent directory to path for imports
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

import integrative_ddm_sim as ddm
import keras
import bayesflow as bf
from bayesflow.networks import SetTransformer, CouplingFlow
from bayesflow.adapters import Adapter
from bayesflow.simulators import make_simulator

from plots import calibration_histogram
from shared.plots import recovery, plot_gamma_recovery_with_mode, compute_recovery_metrics


INFO:bayesflow:Using backend 'jax'


/Users/brunykrijgsman/thesis-ddm/integrative_model


In [3]:
# Load checkpoint - relative to current file
CHECKPOINT_PATH = os.path.join(current_dir, 'checkpoints', 'jax_simple_integrative_ddm_checkpoint_seed12_uniform_new_sigma_beta.keras')

# Create save directory - relative to current file
save_dir = os.path.join(current_dir, 'Figures')
os.makedirs(save_dir, exist_ok=True)


In [4]:
# Define meta function
def meta():
    return dict(n_obs=100)

# Make simulator
print("Making simulator...")
simulator = make_simulator([ddm.prior, ddm.likelihood], meta_fn=meta)

adapter = (
    Adapter()
    .broadcast("n_obs", to="choicert")    
    .as_set(["choicert", "z"])
    .standardize(exclude=["n_obs"])
    .convert_dtype("float64", "float32")
    .concatenate(["alpha", "tau", "beta", "mu_delta", "eta_delta", "gamma", "sigma"], into="inference_variables")
    .concatenate(["choicert", "z"], into="summary_variables")
    .rename("n_obs", "inference_conditions")
)
# Load approximator
approximator = keras.saving.load_model(CHECKPOINT_PATH)

Making simulator...


/Users/brunykrijgsman/miniforge3/envs/tf/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 1 variables whereas the saved optimizer has 303 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
# 10k rows = 100 participants / 100 trials
data = sio.loadmat(os.path.join(current_dir, 'data/integrative_ddm_data_SNR_high_COUP_high_DIST_gaussian.mat'))
nparts = data['nparts'][0][0]
ntrials = data['ntrials'][0][0]
print(f"{data['alpha'].shape}")
print(f"{data['alpha'][0][10]}")
print(f"{data['choicert'].shape}")
all_post_draws = []

# Simulate validation data (unseen during training)
val_sims = simulator.sample(50)
post_draws = approximator.sample(conditions=val_sims, num_samples=50)
print(val_sims.keys())

# (50, 1)
print(val_sims['alpha'].shape)

# (50, 100)
print(val_sims['choicert'].shape)

reshaped_data = {
    'n_obs': np.empty((nparts, 1)),
    'alpha': np.empty((nparts, 1)),
    'tau': np.empty((nparts, 1)), 
    'beta': np.empty((nparts, 1)),
    'mu_delta': np.empty((nparts, 1)),
    'eta_delta': np.empty((nparts, 1)),
    'gamma': np.empty((nparts, 1)),
    'sigma': np.empty((nparts, 1)),
    'choicert': np.empty((nparts, ntrials)),
    'z': np.empty((nparts, ntrials))
}

for npart in range(nparts):
    data_start = npart * ntrials
    data_end = (npart + 1) * ntrials

    # print(f"Sampling for participant {npart+1} of {nparts} ({data_start} to {data_end})")

    reshaped_data['n_obs'][npart] = ntrials
    reshaped_data['alpha'][npart] = data['alpha'][0][npart]
    reshaped_data['tau'][npart] = data['tau'][0][npart]
    reshaped_data['beta'][npart] = data['beta'][0][npart]
    reshaped_data['mu_delta'][npart] = data['mu_delta'][0][npart]
    reshaped_data['eta_delta'][npart] = data['eta_delta'][0][npart]
    reshaped_data['gamma'][npart] = data['gamma'][0][npart]
    reshaped_data['sigma'][npart] = data['sigma'][0][npart]
    reshaped_data['choicert'][npart] = data['choicert'][0][data_start:data_end]
    reshaped_data['z'][npart] = data['z'][0][data_start:data_end]

    # data_subset = {
    #     'n_obs': ntrials,
    #     'alpha': data['alpha'][0][npart],
    #     'tau': data['tau'][0][npart], 
    #     'beta': data['beta'][0][npart],
    #     'mu_delta': data['mu_delta'][0][npart],
    #     'eta_delta': data['eta_delta'][0][npart],
    #     'gamma': data['gamma'][0][npart],
    #     'sigma': data['sigma'][0][npart],
    #     'choicert': data['choicert'][0][data_start:data_end],
    #     'z': data['z'][0][data_start:data_end]
    # }

    # ()   
    # print(data_subset['alpha'].shape)
    # print(data_subset['alpha'])

    # (100,)
    # print(data_subset['choicert'].shape)


# (50, 1)
# print(reshaped_data['alpha'].shape)

# (50, 100)
# print(reshaped_data['choicert'].shape)

post_draws = approximator.sample(conditions=reshaped_data, num_samples=ntrials)
#print(f"post_draws keys: {post_draws.keys()}")



(1, 100)
1.3251421500780303
(1, 10000)
dict_keys(['n_obs', 'alpha', 'tau', 'beta', 'mu_delta', 'eta_delta', 'gamma', 'sigma', 'choicert', 'z'])
(50, 1)
(50, 100)


In [9]:
from shared.plots import recovery, plot_gamma_recovery_with_mode, compute_recovery_metrics

# Prior samples — same way your data was generated:
prior_signs = np.random.choice([-1, 1], size=10000)
prior_gamma = np.random.uniform(2, 3, size=10000) * prior_signs

# Gamma plot
estimates = {
    'gamma': post_draws['gamma']      
}

# plot_gamma_recovery(estimates['gamma'], prior_samples=prior_gamma)
# plt.show()

# Recovery plot
f = recovery(post_draws, reshaped_data)
f.savefig(os.path.join(save_dir, 'recovery_plot_12_uniform150_new_sigma2.png'))
plt.show()


params: ['alpha', 'tau', 'beta', 'mu_delta', 'eta_delta', 'gamma', 'sigma']


AttributeError: 'NoneType' object has no attribute 'savefig'

In [6]:
# Check recovery metrics
compute_recovery_metrics(post_draws, val_sims, prior_samples)

NameError: name 'prior_samples' is not defined

In [7]:
# Get gamma estimates and targets
gamma_estimates = post_draws['gamma']
gamma_targets = reshaped_data['gamma'].flatten()

# Plot recovery
plot_gamma_recovery_with_mode(gamma_estimates, gamma_targets)

# Print sign recovery accuracy
# compute_gamma_sign_recovery(gamma_estimates, gamma_targets)


ValueError: Number of dimensions is greater than number of samples. This results in a singular data covariance matrix, which cannot be treated using the algorithms implemented in `gaussian_kde`. Note that `gaussian_kde` interprets each *column* of `dataset` to be a point; consider transposing the input to `dataset`.

In [ ]:
# Posterior predictive check
# You need to define how to simulate new data given posterior samples
# Check your simulator's interface
# ppc_fig = plot_posterior_predictive_check(
#     make_simulator([ddm.prior, ddm.likelihood], meta_fn=meta), post_draws, val_sims,
#     stat_fn=np.mean,  # or np.std, skew, etc.
#     observed_key="choicert",
# )
# ppc_fig.savefig(os.path.join(save_dir, "posterior_predictive_check.png"))


In [ ]:
# Define variable names explicitly
parameter_names = ["alpha", "tau", "beta", "mu_delta", "eta_delta", "gamma", "sigma"]

# Filter val_sims to only include parameter keys
val_sims_params = {k: v for k, v in reshaped_data.items() if k in parameter_names}

print("Post draws shapes:", {k: v.shape for k, v in post_draws.items()})
print("Val sims params shapes:", {k: v.shape for k, v in val_sims_params.items()})


In [ ]:
# Calibration ECDF plot
ecdf = bf.diagnostics.plots.calibration_ecdf(
    estimates=post_draws, 
    targets=reshaped_data,
    variable_names=parameter_names,
    difference=True,
    rank_type="distance"
)
ecdf.savefig(os.path.join(save_dir, 'calibration_ecdf_12_uniform150_new_sigma2.1.png'))
plt.show()


In [ ]:
# Calibration histogram
sbc = calibration_histogram(
    estimates=post_draws, 
    targets=val_sims_params,
    variable_keys=parameter_names,
    num_bins=10,
    binomial_interval=0.99,
    label_fontsize=16,
    title_fontsize=18
)
sbc.savefig(os.path.join(save_dir, 'calibration_histogram_12_uniform150_new_sigma2.1.png'))
plt.show()
